In [ ]:
import copy
import mne
import math
import numpy as np
import pandas as pd
import pickle
import random


from sklearn import preprocessing
from os.path import exists
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif


In [ ]:
Sampling_Frequency=250
#Length_of_trail=25
k=400
healthy=14
schizo=14
file_name=['h','s']
Band_Pass_Low_Range=0
Band_Pass_High_Range=50
iir_params = {'order': 2, 'ftype': 'butter'}
recording_time=25

In [ ]:
# Random Subjects shuffling
indice=list(range(1,healthy+1))
random.shuffle(indice)
#indice=[6, 11, 10, 1, 8, 3, 13, 5, 7, 12, 9, 14, 4, 2]
indice=[2, 3, 6, 10, 9, 12, 14, 7, 1, 4, 8, 13, 5, 11]
print(indice)

In [ ]:
def select_channel(raw,selected_channel=['Fp1','Fp2']):
    channel_indices=[]
    for channel in selected_channel:
        try:
            index=raw.ch_names.index(channel)
            channel_indices.append(index)
        except ValueError:
            print('Channel{} is not present or again check the names of channel'.format())
    new_raw=raw.get_data(picks=channel_indices)
    return new_raw
            


In [ ]:
def sliding_window_augmentation(data, window_size, stride):
    data=np.array(data)
    n_channels,n_timepoints,  = data.shape
    augmented_data = []
    for start in range(0, n_timepoints - window_size + 1, stride):
        end = start + window_size
        segment = data[:,start:end]
        augmented_data.append(segment)
    return np.array(augmented_data)


In [ ]:


def select_features(new_raw, label, samp_freq=Sampling_Frequency, recording_time=25, augment_ratio=2, k=k):
    scaler = MinMaxScaler()
    trail_length = samp_freq * recording_time   # Calculating Length of each Trail
    stride = trail_length // augment_ratio
    trail_length_shape = np.shape(new_raw)[1]   # Determining the time of each channel
    total_chunks = trail_length_shape // trail_length    # Final How many chunks can be created 
    new_raw = new_raw[:, 0:total_chunks * trail_length]     # Changing Shape of Data
    augmented_eeg_data = sliding_window_augmentation(new_raw, trail_length, stride)
    
    feature = []
    for chunk in augmented_eeg_data:
        segg = []
        for i in range(chunk.shape[0]):
            fourier = np.abs(np.fft.fft(chunk[i, :]))
            # Select the top k features based on their scores
            selected_features = SelectKBest(score_func=f_classif, k=k)
            selected_features.fit_transform(fourier.reshape(1, -1), np.array([label]))  # Assuming label is binary
            selected_indices = selected_features.get_support(indices=True)
            selected_fourier = fourier[selected_indices]
            segg.append(selected_fourier)
        final_concate = np.concatenate(segg, axis=0)
        final_concate = final_concate.reshape(-1, 1)
        final_concate = scaler.fit_transform(final_concate)
        final_concate = final_concate.ravel()
        feature.append((final_concate, label))
    return feature


In [ ]:
def equal_length(pos_class_train,neg_class_train):
    if len(pos_class_train) > len(neg_class_train):
        pos_class_train = pos_class_train[:len(neg_class_train)]
    elif len(pos_class_train) < len(neg_class_train):
        neg_class_train = neg_class_train[:len(pos_class_train)]
    return pos_class_train,neg_class_train


In [ ]:
# Combining all and preprocessing the whole data
def preprocess(subject_name,indices,labels,number_of_subjects=14,train_percent=0.8,Band_Pass_Low_Range=0,Band_Pass_High_Range=50,trail_times=25,augmented_ratio=2):
    limit=math.ceil(train_percent*number_of_subjects)
    k=1
    train_data=[]
    test_data=[]
    for subject_number in indices:
        if exists('./Dataset/{}{}.edf'.format(subject_name,subject_number)):
            file='./Dataset/{}{}.edf'.format(subject_name,subject_number)
            raw=mne.io.read_raw_edf(file,verbose=0,preload=True,eog=None,exclude=(),stim_channel='auto')
            raw=raw.pick_types(eeg=True)
            raw=raw.filter(Band_Pass_Low_Range,Band_Pass_High_Range,iir_params=iir_params,method='iir')
            new_raw=select_channel(raw)
            #new_raw=normalize_eeg_data(new_raw)
            chunks=select_features(new_raw,label=labels,recording_time=trail_times,augment_ratio=augmented_ratio)
            if k<=limit:
                train_data+=chunks
            else:
                test_data+=chunks
            k=k+1
    return train_data,test_data



In [ ]:
pos_class_train,pos_class_test=preprocess('h',indices=indice,labels=0,trail_times=recording_time,augmented_ratio=1,train_percent=0.8)
neg_class_train,neg_class_test=preprocess('s',indices=indice,labels=1,trail_times=recording_time,augmented_ratio=1,train_percent=0.8)
pos_class_test,neg_class_test=equal_length(pos_class_test,neg_class_test)
testing_data=pos_class_test+neg_class_test

In [ ]:
# Using double backslashes
with open('./samples/pos_class_train.pkl', 'wb') as f:
    pickle.dump(pos_class_train, f)

# Using double backslashes
with open('./samples/neg_class_train.pkl', 'wb') as f:
    pickle.dump(neg_class_train, f)

with open('./samples/testing_data.pkl', 'wb') as f:
    pickle.dump(testing_data, f)